### How does a playlist’s number of tracks, average song length, and total duration affect its popularity?

In [16]:
import os
import json
import pickle
from collections import defaultdict

In [17]:
MPD = 'mpd/'
playlist_files = [os.path.join(MPD, x) for x in os.listdir(MPD)]

In [18]:
SONG_DATA = 'mappings/songs'
song_files = [os.path.join(SONG_DATA, x) for x in os.listdir(SONG_DATA)]

In [19]:
songID_to_duration = {}
for file in song_files:
    with open(file, 'rb') as fp:
        data = pickle.load(fp)
    for k,v in data.items():
        songID_to_duration[k] = v['duration_ms']

In [20]:
list(songID_to_duration.items())[:5]

[('spotify:track:1fyTBapjw8q9MlzLVdU6wg', 231986),
 ('spotify:track:4cC9nQMgClmz8cix6l5CSX', 306127),
 ('spotify:track:3qwh1awyjkXFUER1Pp64qv', 283040),
 ('spotify:track:6NHONOfLaDTtSG8WtCTpex', 501626),
 ('spotify:track:7HxecasMeh6aCAvQPGiFgP', 346666)]

In [21]:
with open('mappings/Q2/playlist_dataAUTHORCOUNT.pkl', 'rb') as fp:
    pid_to_authors = pickle.load(fp) 

In [22]:
pid_to_authcount = defaultdict(int)
for k,v in pid_to_authors.items():
    s = set()
    for song in v[0]:
        s.update(song)
    pid_to_authcount[k] = len(s)

In [23]:
# IDEA: can also look at playlist_num['num_albums']
playlist_data = {}
for file in playlist_files:
    with open(file, 'rb') as fp:
        data = json.load(fp)
    for playlist in data['playlists']:
        ID = playlist['pid']
        followers = playlist['num_followers']
        durations = []
        for track in playlist['tracks']:
            if track['track_uri'] in songID_to_duration:
                duration = songID_to_duration[track['track_uri']]
                durations.append(int(duration))
        number_of_tracks = len(durations) # since some might be None, take this isntead of data parameter
        total_duration = sum(durations)
        avg_song_length = total_duration/number_of_tracks
        albums = playlist['num_albums']
        total_author_count = pid_to_authcount[ID]
        playlist_data[ID] = (number_of_tracks, total_duration, avg_song_length, albums, total_author_count, 
                             followers)

In [24]:
list(playlist_data.items())[:5]

[(962000, (151, 43501164, 288087.178807947, 58, 49, 1)),
 (962001, (143, 29174668, 204018.65734265733, 40, 25, 1)),
 (962002, (75, 16744221, 223256.28, 58, 89, 1)),
 (962003, (32, 7621816, 238181.75, 32, 61, 1)),
 (962004, (27, 6261480, 231906.66666666666, 27, 31, 1))]

In [25]:
# id: num_tracks, total_duration (ms), avg_song_length (ms), followers
with open('mappings/Q3/playlist_datav3.pkl', 'wb') as fp:
    pickle.dump(playlist_data, fp)